In [38]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go

from ipywidgets import Select, Output, Button,Box
from IPython.display import display

import requests

# Define callback functions

Main callback

In [40]:
output = Output()

@output.capture()
def callback(self):
    output.clear_output()
    print( "Selection is {}".format(state_widget.get_interact_value()))
    covid_json = get_data(state_widget.get_interact_value())
    geos,center = process_data(covid_json)
    df = json_to_dataframe(covid_json)
    print("Got data. Rendering...")
    fig = plot(df,geos,center)
    display(go.FigureWidget(fig))

## Get the data

In [26]:
def get_data(state):
    rest_url = 'https://services9.arcgis.com/6Hv9AANartyT7fJW/ArcGIS/rest/services/USCounties_cases_V1/FeatureServer/0/query?where=ST_Abbr%3D%27{}%27&objectIds=&time=&geometry=&geometryType=esriGeometryPolygon&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&returnGeodetic=false&outFields=Countyname%2CST_Name%2CConfirmedb%2CConfirmed%2CPOP_ESTIMA&returnGeometry=true&returnCentroid=false&featureEncoding=esriDefault&multipatchOption=none&maxAllowableOffset=&geometryPrecision=&outSR=3819&datumTransformation=&applyVCSProjection=true&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pjson&token='.\
            format(state)

    response = requests.get(rest_url)
    return response.json()

## Process data

Get data into a format that can be used by Plotly.

In [5]:
def process_data(covid_json):
    geo_features = []
    lat_longs = []
    for rec in covid_json['features'] :
        new_rec = {}
        new_rec['properties'] = rec['attributes']
        new_rec['geometry'] = {'type' : 'Polygon' }
        new_rec['geometry']['coordinates'] = rec['geometry']['rings']
        lat_longs.append(new_rec['geometry']['coordinates'][0][0])
        geo_features.append(new_rec)

    geos = { "type" : "FeatureCollection", "features" : geo_features}
    # Calculate center
    center = pd.DataFrame(lat_longs,columns=['lon','lat']).mean().to_dict()
    return geos,center

## Convert to data frame

In [6]:
def json_to_dataframe(covid_json):
    features = [ x['attributes'] for x in covid_json['features']]


    df = pd.DataFrame(features)
    df ['density'] = df.Confirmed / df.POP_ESTIMA * 1_000
    return df

## Plot

In [7]:
def plot(df,geos,center):
    fig = px.choropleth_mapbox(df,geojson=geos,
                           featureidkey='properties.Countyname',
                           locations='Countyname',
                           color='density',
                           mapbox_style="carto-positron", 
                           center=center, 
                           #opacity=0.5,
                           zoom=5,
                           hover_data=['Confirmed','POP_ESTIMA'],
                           #title='COVID cases per capita',
                           #height=650
                          )
    return fig

# Output

Render output here

In [41]:
output

Output()

## Select state

In [42]:
states = pd.read_csv("state_abbrv.tsv",sep="\t").set_index('state').abbreviation.to_dict()
state_widget = Select(description='Select a state', options=states,rows=10)
button = Button(description="go")
button.on_click(callback)
Box([state_widget,button])

Box(children=(Select(description='Select a state', options={'Alabama ': 'AL', 'Alaska ': 'AK', 'Arizona ': 'AZ…